# 7 Analysis Step #5; Annovar

<!-- http://nar.oxfordjournals.org/content/38/16/e164 -->

In the - for now - final Galaxy workflow step, we are going to annotate all our filtered variants to determine the severity of the variations we determined last time with the `Varscan` tool. This involves comparing all our variants against a number of databases containing *annotation data* for known variants. We will use our filtered `VCF`-file resulting from Varscan so we need to upload our newly created VCF file (the output from `deliverable8.py`) to Galaxy, if you do not (yet) have your own version of this tool available in Galaxy. Make sure that Galaxy recognizes the file type by selecting the correct file type when uploading or change it afterwards using the *Edit attributes* button.

<!--
<strong>NOTE:</strong> In the previous step we removed the first 24 lines from the vcf file. These first 24 lines are actualy needed by this step. So before we proceed, we should rerun our previuous script and not delete those lines, but write them also to the output file. You can upload your filtered vcf file to galaxy to work on. Select the <strong>upload button</strong> which is in the upper left corner next to the <strong>Tools</strong>. Select the <strong>Choose local file</strong>. Set the type to <strong>vcf</strong> and the genome to <strong>hg19[Human feb. 2009(GRCh37/hg19)]</strong>
-->
<img src="pics/upload.png">

## 7.1 Running Annovar

Select the [**ANNOVAR**](http://annovar.openbioinformatics.org/en/latest/) tool by searching for its name or selecting it from the *Diagnostic Genome Analysis* tool menu. This tool offers a lot of configuration options in the form of selectable sources for comparison. Note that not all of these sources are actually available in our Galaxy server, so take care when selecting the proposed options below.

Set the following settings and data sources from the tool configuration as follows:
* **Reference**: `hg19` [Human Feb. 2009 (GRCh37/hg19)]
* **Select file to annotatte**: your uploaded **filtered** VCF file
* **Select filetype**: `VCF4 file`
* **Select Gene Annotations**: `Select all`
* **Select dbSNP version(s) to annotate with**: `138 (hg19 only)`
* **Select 1000Genomes Annotation**: `2015aug (hg19/hg38) (6 populations: AMR,AFR,EUR,EAS,SAS,ALL)`
* **Select Exome Variant Server version(s) to annotate with**: `ESP6500si European Americans`
* **Annotate with ExAC 03?**: `Yes`
* **CLINVAR Annotation? (hg19 only)**: `Yes`
* **Select functional impact scores (LJB2)**: `Select all`
* **Also get predictions where possible?**: `Yes`

With all these options selected, Execute the tool.

<img src="pics/annovar.png">

Download the resulting `.tabular` file to your computer and open it with Excel or the Linux alternative (you might need to change the file extension to `.tsv` (for *tab separated values* format) for Excel to recognize the file). As you will see, the file looks like the input-VCF file, but a number of columns are added containing the annotation data from all the selected data sources. 
* Create a table in your report where you summarize the added columns (very!) briefly;
    * The database where the data comes from
    * The result or value (is it an identifier, a percentage, etc.). See the links in the overview below.

## 7.2 Documentation; Writing an Introduction

As we've now run the final step in our analysis pipeline we *should* know everything about the process going from raw sequencing reads (the FASTQ file contents) to annotated variants. If *any* of the previous steps is not yet clear you should revisit those step(s) and carefully re-read the documentation.

With our current knowledge you should be able to write a complete **Introduction** chapter, please see the instructions and the report template available on [Blackboard](https://blackboard.hanze.nl/webapps/blackboard/content/listContentEditable.jsp?content_id=_2558467_1&course_id=_5800_1). Create a new 'Introduction' document in your (shared) OneNote folder and start writing this chapter.

The Introduction chapter is also a common place to reference to books, web-resources and publications which is why you are also asked to write a **References** chapter (in a new document). Please carefully read the instructions in the template report as this contains examples for all three mentioned resources. Also note that most Galaxy tools are published and they contain a *how-to-cite* link (or even an export button) which you can use to compile your list.

## 7.3 Programming Assignment; Analyzing Variant Annotation

From all the annotation columns we need to get the following data for each variant. 
Note: for some of these data sources only a few of your variants are actually annotated
Note: the list below links to the Annovar documentation regarding this value; read these documents to at least *know* what they mean.
* The gene name from the `RefSeq_Gene` column,
* the gene 'function' from the `RefSeq_Func` column (see the [table of possible values](http://annovar.openbioinformatics.org/en/latest/user-guide/gene/#output-file-1-refseq-gene-annotation) for this column),
* the [`dbsnp`](http://annovar.openbioinformatics.org/en/latest/articles/dbSNP/) identifier,
* the 1000-genomes data for the `EUR` population group (the others are not of interest),
* the [`sift`](http://annovar.openbioinformatics.org/en/latest/user-guide/filter/#-sift-annotation) value from the `LJB2_SIFT` column,
* the [`polyphen`](http://annovar.openbioinformatics.org/en/latest/user-guide/filter/#-polyphen-2-annotation) value from one of the `LJB2_PolyPhen2_*` columns and
* the [`clinvar`](http://annovar.openbioinformatics.org/en/latest/user-guide/filter/#clinvar-annotations) value.

To make proper use of this annotation data we will once again write a *filter* Python program that *parses* the ANNOVAR output file and filters out these values. Instead of writing this to a new file, we will store the results into a MySQL database consisting three linked tables.

<!-- MINOR EDIT
of three tables as shown below:

<img src="pics/Tablet.png">
<img src="pics/anno-db.png">
-->

This programming assignment consists of four deliverables (9, 10 and 11) where the first one focuses on parsing the ANNOVAR output file (and simply prints a number of values per variant). Deliverable 10 is for creating a *regular-expression pattern* for parsing the gene name from the `RefSeq_Gene` column. Deliverable 11 asks to create a database schema for storing all relevant information and deliverable 11 is for inserting the parsed data into the this database.

### Deliverable 9; ANNOVAR file parsing

Given the list of required fields from the ANNOVAR output file, write a program that reads in an ANNOVAR output file (you can use the following [example data (`example.txt`)](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/data/example.txt) to 'extract' this data.

Notes:
* Pre-define the columns to extract, i.e. `columns = [10, 11, 15:18, 20:24]`
* Combine this with the parsed *header* string (`"chromosome\tbegin\tend\treference\tobserved\tCHROM\tPOS..."`)
* Output of this deliverable should be a **printed** summary of the data for **each** variant:
    * A single line giving a variant *position* overview, i.e. gene name, chromosome, reference base, etc.
    * A set of **indented** lines, with one of the above annotation values per line (i.e. `"\tdbsnp: rs1621733"`)
    * A blank line before the next variant


<!--based on the [Deliverable 9 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable9.py) Nope... -->

### Deliverable 10; Gene-name parsing

The `RefSeq_Gene` column in the ANNOVAR output file contains a somewhat complex representation of the gene name that the variant is associated with. It could contain just the text '`ERBB4`' if the variant lies on an exon of that gene, but it could also be '`BIN1(dist=32600),CYP27C1(dist=43909)`' if the variant lies in the intergenic region *between* the `BIN1` and `CYP27C1` genes where the `dist=***` denotes the distance to these genes. This assignment filteres out the gene names, and thus removes these distance numbers.

Create a regular expression pattern that catches all gene names from the sometimes complex value of the `RefSeq_Gene` column. The [Deliverable 10 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable10.py) contains a function that gets the value from this annotation column and returns the actual gene name(s) by 'extracting' it using a regular expression.

Example data has been made available at the online regular expression test website [regex101](https://regex101.com/r/rmxDs5/2/) and click on the **fork regex** button to start designing your own regular expression. The `Test String` field contains seven lines taken from the `RefSeq_Gene` column that you can use to test your pattern on. You are free to add data to this field from your own ANNOVAR output file.

The following block shows the example input data and the expected output data. As you can see, we are only interested in retaining the gene name and not the `NONE`, `LOC****` and `LIN****` fields. Please read the instructions in the template script carefully for further details.

```
'TNNI3(NM_000363:exon5:c.371+2T>A)'                      ----> 'TNNI3'
'TSHZ3(dist=65732),THEG5(dist=173173)'                   ----> 'TSHZ3/THEG5'
'ACTR3BP2(dist=138949),NONE(dist=NONE)'                  ----> 'ACTR3BP2'
'BIN1(dist=32600),CYP27C1(dist=43909)'                   ----> 'BIN1/CYP27C1'
'LOC101927282(dist=1978702),LINC01519(dist=14658)'       ----> '-'
'NBPF10,NBPF20'                                          ----> 'NBPF10/NBPF20'
'ERBB4'                                                  ----> 'ERBB4'
'LOC100507291'                                           ----> '-'
'NONE'                                                   ----> '-'
```

### Deliverable 11; Database Schema

Create an SQL database schema that consists of three tables to store information from the (*i*) Chromosomes, (*ii*) Genes and (*iii*) Variants. These tables should have a relation making it possible to i.e. get all variants for a specific gene (by name) or count the number of variants per chromosome etc.

Carefully study the output of the Annovar tool to decide where you can find the information and to define the datatype to use for storage.

For this assignment you are asked to create both a graphical representation of your database (image, place this in your notebook) and an SQL file containing the `CREATE TABLE` statements (add this file to your forked repository). Your SQL file should begin with one or more [`DROP TABLE ... IF EXISTS`](https://dev.mysql.com/doc/refman/5.7/en/drop-table.html) 

### Deliverable 12; Database Interaction

With the ANNOVAR file parsed and the gene names set correclty it is time to create and fill the database. Your program should have the following functionality:

* Accept the following input parameters using the `argparse` Python library:
    + Database `name`
    + Database `username`
    + Database `password`
    + ANNOVAR annotation file
    + (optional) database schema file (`.sql` file for re-creating the database)
* **Test** the given database name and credentials (i.e. perform a simple [`SELECT 1`](https://stackoverflow.com/questions/3668506/efficient-sql-test-query-or-validation-query-that-will-work-across-all-or-most) query)
* **Test** if the given ANNOVAR file exists and if it contains tab-separated data (i.e., the length of the tab-splitted header should be > 1)
* If given **and** the previous test(s) pass, execute the `.sql` file (this will `DROP` all existing table(s))
* Parse the ANNOVAR file (use the code from deliverable 9)
    * Either build up a proper object containing the data and logic for parsing (see 'bonus' below) or
    * Process the parsed data on-the-fly;
        * Once you have the chromosome information; insert into the `Chromosome` table,
        * Once you have the gene information (parsed by your deliverable 10 code), insert into the `Gene` table,
        * Once you have the variant information, insert into the `Variant` table.
* Once all data has been parsed and inserted into the database, a small report should be printed that states the number of rows in each table. This should be added to your OneNote journal file.

The program should also properly report any problems encoutered.

* Bonus points available for:
    * Proper use of OO-techniques
    * Dividing coherent functionality accross multiple modules



<!-- The [Deliverable 12 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable12.py)

Please read the instructions carefully for the `parse_annovar_data` function from deliverable 9 since we now need to replace our `print()` statements with *calls* to the functions that will fill the database. The comment in the `parse_annovar_data` function tells you which part to copy and which part(s) to create.

The `create_database` function should contain all code needed for creating the tables using `CREATE TABLE` statements. This function requires two parameters; the database `connector` and the database `cursor`. Both are created in the `main()` function and *passed* to ALL functions that interact with the database:
```
conn = sqlite3.connect(sqliteDB)
cursor = conn.cursor()
```
Before creating the tables though, it is custom to first try to remove the tables if they already exist, using `DROP TABLE` statements. Follow the instructions given in the function comments.
-->

## 7.4 Finding Variants of Interest

This section is still in writing